In [11]:
import ast
import glob
import numpy as np
import pandas as pd

In [2]:
files = glob.glob('temp/mastodon_users_papers_*.tsv')

In [3]:
files[:3]

['temp/mastodon_users_papers_2023-08-05 02:04:54.290566.tsv',
 'temp/mastodon_users_papers_2023-08-05 02:33:59.793183.tsv',
 'temp/mastodon_users_papers_2023-08-05 02:51:33.621818.tsv']

In [4]:
len(files)

13

In [5]:
data = []
for file in files:
    data.append(pd.read_csv(file, sep='\t'))
    
data = pd.concat(data)

In [6]:
len(data)

123007

In [10]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,id,authorships:author:id,publication_year,referenced_works
0,2239417385,"[4355774393, 4355684289, 2123179122]",2015,[]
1,1507194541,"[4358415098, 4357563902, 4342583151, 174528948...",2009,[]
2,78935132,[112741616],1988,[]
3,2130771758,"[77456698, 1980551004]",2010,"[7641444,9210600,10541243,12276110,15429805,27..."
4,3134907177,[2156773890],2017,[]


In [17]:
from collections import defaultdict

authors_year = defaultdict(lambda:[])
for idx, row in data.iterrows():
    authors = ast.literal_eval(row['authorships:author:id'])
    year = row['publication_year']
    for author in authors:
        authors_year[author].append(year)

In [18]:
len(authors_year)

399414

In [20]:
data = pd.read_csv('kuni_twitter_open_alex_mstd_07-18-2023.tsv', sep='\t')
data = data.drop(columns=['Unnamed: 0'], axis=1)

# ntoots = pd.read_csv('kuni_ntoots.tsv', sep='\t')
# data = data.merge(ntoots, on='mastodon_name')
data.head()

,displayname,followers,following,bio,mastodon_name,OpenAlex_account,x_concepts_x,works_count,cited_by_count,most_cited_work,summary_stats,counts_by_year
0,Denis Duboule,342,66,<p>Geneticist in transit. Collège de France/EP...,DenisDuboule@mas.to,https://openalex.org/A295410851,"[{'id': 'https://openalex.org/C54355233', 'wik...",2,154,Engineering chromosomes in mice through target...,"{'2yr_mean_citedness': 0, 'h_index': 1, 'i10_i...","[{'year': 2023, 'works_count': 1, 'oa_works_co..."
1,Jon Hughes,402,842,<p>Academic (Reader in German/Cultural Studies...,rider_jon@zirk.us,https://openalex.org/A2646935175,"[{'id': 'https://openalex.org/C142362112', 'wi...",2,0,Weltfabrik Berlin: Eine Metropole als Sujet de...,"{'2yr_mean_citedness': 0, 'h_index': 0, 'i10_i...",[]
2,Magnus Palmblad,141,119,<p>Scientist. Skeptic. Swede. Not the vegetabl...,magnuspalmblad@fediscience.org,https://openalex.org/A2058370087,"[{'id': 'https://openalex.org/C86803240', 'wik...",11,7,Identification and Characterization of Peptide...,"{'2yr_mean_citedness': 0, 'h_index': 2, 'i10_i...","[{'year': 2023, 'works_count': 2, 'oa_works_co..."
3,João Ohara,224,147,<p>Theory and Philosophy of History @ Federal ...,ohara@hcommons.social,https://openalex.org/A2284742525,"[{'id': 'https://openalex.org/C138885662', 'wi...",24,7,Virtudes Epistêmicas na Prática do Historiador...,"{'2yr_mean_citedness': 1.0, 'h_index': 2, 'i10...","[{'year': 2022, 'works_count': 0, 'oa_works_co..."
4,Neil Wright,56,106,<p>Medical Statistician. Topology and geometry...,neilstats@fediscience.org,https://openalex.org/A2576451780,"[{'id': 'https://openalex.org/C15744967', 'wik...",1,1,The role of cluster size and intra-cluster cor...,"{'2yr_mean_citedness': 0, 'h_index': 1, 'i10_i...","[{'year': 2022, 'works_count': 0, 'oa_works_co..."


In [32]:
BIRTH = []
NPAPERS = []
for idx, row in data.iterrows():
    author_openalex_id = int(row['OpenAlex_account'][22:])
    papers = authors_year[author_openalex_id]
    if len(papers) > 0:
        birth = min(papers)
        npapers = len(papers)
    else:
        birth = None
        npapers = None
    
    BIRTH.append(birth)
    NPAPERS.append(npapers)

data['birth_openalex'] = BIRTH
data['npapers_openalex'] = NPAPERS

In [ ]:
for k,v in authors_year.items():
    print(k, v)

In [34]:
data.to_csv('kuni_authors_with_open_alex.tsv', sep='\t')